<a href="https://www.kaggle.com/code/yassinezarwal/final-titanic?scriptVersionId=97831667" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [103]:
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
    
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.pipeline import  Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler , OrdinalEncoder ,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split ,cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression ,SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_regression
from sklearn.svm import SVC
from sklearn.tree import export_graphviz
from sklearn.metrics import accuracy_score


/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [104]:
rfc=RandomForestClassifier()


best_rfc=GridSearchCV(rfc,{'n_estimators':[60,70,80],
                                 
#                                  'min_samples_split':[3,4,5],
#                            'min_samples_leaf':[3,4,5],
                           'max_depth':[5],
#                            'random_state':[1]
                                
                                },cv=5)
# ,n_iter=30
# the most important parametre is max depth , min sample and n estimators isnt

In [105]:
data_train=pd.read_csv('/kaggle/input/titanic/train.csv',index_col='PassengerId')
data_test=pd.read_csv('/kaggle/input/titanic/test.csv',index_col='PassengerId')
# train_test_split
train_y=data_train.Survived    
train_x=data_train.drop(columns=['Ticket','Name','Cabin','Survived' ])

test_x=data_test.drop(columns=['Name','Ticket','Cabin'])

cat_col=train_x.select_dtypes('object').columns.tolist()
num_col=train_x.select_dtypes(['float','int']).columns.tolist()

train_x=train_x[num_col+cat_col]
test_x=test_x[num_col+cat_col]
print(train_x.head())

             Pclass   Age  SibSp  Parch     Fare     Sex Embarked
PassengerId                                                      
1                 3  22.0      1      0   7.2500    male        S
2                 1  38.0      1      0  71.2833  female        C
3                 3  26.0      0      0   7.9250  female        S
4                 1  35.0      1      0  53.1000  female        S
5                 3  35.0      0      0   8.0500    male        S


In [106]:

pipline_num=Pipeline([('imputer_num',SimpleImputer(strategy='mean'))])
pipline_cat=Pipeline([('imputer_cat', SimpleImputer(strategy='most_frequent')),('encoder',OrdinalEncoder())])
full_pipline=ColumnTransformer([('num',pipline_num,num_col),('cat',pipline_cat,cat_col)])
train_x_final=pd.DataFrame(full_pipline.fit_transform(train_x),index=train_x.index,columns=train_x.columns)
test_x_final=pd.DataFrame(full_pipline.fit_transform(test_x),index=test_x.index,columns=train_x.columns)
ind=test_x_final.index

In [107]:
train_x,train_test_x,train_y,train_test_y=train_test_split(train_x_final,train_y,test_size=0.2)




In [108]:
    best_rfc.fit(train_x,train_y)
    new_model=best_rfc.best_estimator_
    print('best pra',best_rfc.best_params_)
    print(best_rfc.best_score_)

    print('best model is : ',new_model)
    p=new_model.predict(train_test_x)
    print(accuracy_score(train_test_y,p))

best pra {'max_depth': 5, 'n_estimators': 80}
0.8202797202797203
best model is :  RandomForestClassifier(max_depth=5, n_estimators=80)
0.8212290502793296


In [109]:

# export_graphviz(
# # new_model,
# # out_file=None,
# # feature_names=train_x.columns,
# # class_names=[0,1],
# # rounded=True,
# # filled=False ,
# )

In [111]:
predictions=pd.DataFrame(new_model.predict(test_x_final),columns=['Survived'],  index=test_x_final.index)
print(predictions)
predictions.to_csv('prediction9.csv')

             Survived
PassengerId          
892                 0
893                 0
894                 0
895                 0
896                 1
...               ...
1305                0
1306                1
1307                0
1308                0
1309                0

[418 rows x 1 columns]
